<img src="https://i.stack.imgur.com/GxY2t.png" align="right" width="200">
I end up using [geopandas]() on a regular basis, and one of its minor irritants is getting the unique number of geometries in a GeoDataFrame.

In this particular instance, I want to know how many duplicate locations I had in my dataset. The solution involves a way to sidestep the issue. In it, we'll get to learn the difference between mutable and immutable objects in Python, and their various properties.


## Background

In [44]:
import geopandas as gpd
from shapely.geometry import Point

In [45]:
# Immutable objects are hashable, e.g. strings
print(set(['apple', 'orange']))

{'apple', 'orange'}


In [46]:
# But not mutable ones
print(set(['apple', ['orange', 'banana']]))

TypeError: unhashable type: 'list'

That's because we're feeding `set()` with a string and a list, the latter being a mutable object. Lists are meant to change readily (as opposed to tuples), and Python disallows their serialization when trying to hash them.

## The problem
Geopandas geometries are stored as `shapely.geometry` objects, which have the interesting following attribute of being unhashable in Python.

In [47]:
points = [Point([0, 0]), Point([1, 1])]
print(len(set(points)))

TypeError: unhashable type: 'Point'

Similarly with Geopandas, which stores its `geometry` column as `shapely.geometry` objects:

In [48]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
print(len(world['geometry'].unique()))

TypeError: unhashable type: 'Polygon'

The reason for this is because shapely objects are mutable &mdash; each object is meant to be transformed and rewritten-to in memory under the same variable name.



## The solution
To get around this issue, you need to modify the way the `geometry` object is represented before passing your iterable to `set()` or `GeoSeries.unique()`.

Like above, we'll change them to type `str`, which are immutable. 

In [49]:
geometries = world['geometry'].apply(lambda x: x.wkt).values
print(geometries[0][:100], '...')

POLYGON ((61.21081709172574 35.65007233330923, 62.23065148300589 35.27066396742229, 62.9846623065766 ...


In [50]:
len(set(geometries))

177

## See also
See [discussion on the project's Github repo](https://github.com/Toblerity/Shapely/pull/320), where the issue is discussed with the owner of the package:

>[sgillies](https://github.com/sgillies) commented on Sep 6, 2015

>Shapely's geometries are mutable, but we're providing a hash function. These two features are inconsistent. Rather than remove mutability (for now) we'll remove the hashability.

Also of note, the issue is also [discussed in geopandas issue 221](https://github.com/geopandas/geopandas/issues/221). In some ways, this behaviour jars with the rest of pandas, which provide "view access" to the data is stores... immutably.

A word to the wise!